In [1]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

[ Info: Precompiling LowLevelFEM [6171b9fb-adbf-4751-adb9-5faded75de07] (cache misses: include_dependency fsize change (2), wrong dep version loaded (2), incompatible header (14))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


In [2]:
gmsh.open("body2.geo")

Info    : Reading 'body2.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.00178275s, CPU 0.001782s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000646868s, CPU 0.000618s)
Info    : Meshing 3D.

In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:HeatConduction)

Problem("body2", :HeatConduction, 3, 1, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 3321, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [4]:
supp = FEM.temperatureConstraint("left", T=0)
load = FEM.heatFlux("right", qn=-1)

BoundaryCondition("right", nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, -1, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [5]:
T0 = FEM.initialTemperature(problem, "body", T=0)

ScalarField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [6]:
FEM.initialTemperature!(T0, "left", T=0)
T0

ScalarField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [7]:
K = FEM.heatConductionMatrix(problem)
C = FEM.heatCapacityMatrix(problem)
q = FEM.heatFluxVector(problem, [load])

ScalarField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [9]:
λmax = FEM.largestEigenValue(K, C, support=[supp])

29474.62441063031

In [11]:
ϑ = 0.33
T = FEM.FDM(K, C, q, T0, 200, 10 * 2 / ((1 - 2ϑ) * λmax), ϑ=ϑ)

ScalarField
[0.0 -2.612889224414745e-28 … -8.060979294111713e24 1.5499172198840492e25; 0.0 -2.6128656092371852e-28 … -6.068190886626717e24 1.1701245181220406e25; … ; 0.0 -1.2486392161077963e-5 … 2.3044087791963978e35 -4.345739876139133e35; 0.0 -0.0002388769758174594 … 4.995376277786303e35 -9.414962081546403e35]

In [12]:
ϑ = 0.66
T = FEM.FDM(K, C, q, T0, 15, 0.01, ϑ=ϑ)

ScalarField
[0.0 -8.362840625823422e-17 … -1.49472649134504e-8 -3.089866586656398e-8; 0.0 -8.362840625823436e-17 … -1.4947264913450428e-8 -3.089866586656404e-8; … ; 0.0 -0.0008335803349351795 … -0.019271366519329324 -0.020440644855610736; 0.0 -0.004365347787506133 … -0.028044930838098246 -0.029298012153639124]

In [13]:
FEM.showDoFResults(T, :T, visible=true)

1

In [14]:
q1 = FEM.solveHeatFlux(T)

VectorField
[[0.0 -2.2811731601893962e-15 … -6.514900471659904e-8 -1.1440433533797527e-7; 0.0 2.129924444096732e-28 … -2.3822801641527197e-21 -4.7645603283054394e-21; … ; 0.0 -8.874685183736383e-30 … -5.955700410381799e-22 1.1911400820763599e-21; 0.0 8.874685183736383e-30 … 0.0 0.0], [0.0 1.569905205886489e-14 … 1.6558069221101488e-6 3.1904507307056267e-6; 0.0 8.164710369037472e-28 … 9.529120656610879e-21 1.4293680984916318e-20; … ; 0.0 1.7749370367472766e-29 … 0.0 4.7645603283054394e-21; 0.0 1.7749370367472766e-29 … -1.1911400820763599e-21 2.3822801641527197e-21], [0.0 8.749278718833417e-14 … 5.399022587674773e-6 1.004844528070696e-5; 0.0 5.537803554651503e-27 … 4.7645603283054394e-20 9.529120656610879e-20; … ; 0.0 2.129924444096732e-28 … 1.6675961149069038e-20 0.0; 0.0 1.7582588197507259e-28 … 2.9365169224414203e-20 3.0407258382252453e-20], [0.0 4.591959660724372e-13 … 1.54204957864038e-5 2.753384751870066e-5; 0.0 2.4423133625642525e-26 … 5.717472393966527e-20 1.1434944787933055e-19;

In [15]:
FEM.showHeatFluxResults(q1, :qvec)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


2

In [16]:
q2 = FEM.solveHeatFlux(T, DoFResults=true)

VectorField
[0.0 -2.2811731601893075e-15 … -6.514900471659785e-8 -1.1440433533796813e-7; 0.0 -9.762153702110021e-29 … -4.7645603283054394e-21 -4.7645603283054394e-21; … ; 0.0 0.0 … 1.8735013540549517e-15 6.245004513516506e-16; 0.0 -1.5612511283791264e-16 … -1.8735013540549517e-15 -2.4980018054066022e-15]

In [17]:
FEM.showDoFResults(q2, :qvec)

3

In [18]:
q3 = FEM.elementsToNodes(q1)

VectorField
[0.0 -2.2811731601893075e-15 … -6.514900471659785e-8 -1.1440433533796813e-7; 0.0 -9.762153702110021e-29 … -4.7645603283054394e-21 -4.7645603283054394e-21; … ; 0.0 0.0 … 1.8735013540549517e-15 6.245004513516506e-16; 0.0 -1.5612511283791264e-16 … -1.8735013540549517e-15 -2.4980018054066022e-15]

In [19]:
FEM.showDoFResults(q3, :qvec)

4

In [20]:
err1 = FEM.fieldError(q1)

VectorField
[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

In [21]:
FEM.showDoFResults(err1, :vector)

5

In [22]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0
Fontconfig warning: using without calling FcInit()


In [23]:
gmsh.finalize()